# Data Cleaning


Lesson Goals

    Examine data for potential issues.
    Identify and fill in missing values.
    Identify and correct incorrect values.
    Remove low variance columns.
    Identify potential outliers.
    Correct incorrect data types.
    Remove special characters and clean categorical variables.
    Identify and remove duplicate records.

Introduction

When working with data sets, you will find that they often require a bit of cleaning. Whether Pandas originally read the data types incorrectly, records are duplicated, the data contains special characters or missing value, or there are slightly different references to the same entity, every data analyst must know how to clean the data they are working with before analyzing it. In this lesson, you will learn about some of the most common problems that make data messy and methods for correcting those problems and cleaning your data.

The data set we are going to be using for this lesson is a messy version of the vehicles data set we worked with in the previous lesson. Let's import this version of our data set so that we can then practice cleaning it up.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('vehicles_messy.csv')

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Examining Data for Potential Issues

One of the first things we want to do is examine the data and look for any potential issues. Some of the things we are interested in identifying in the data at this stage include:

    Missing values
    Special characters
    Incorrect values
    Extreme values or outliers
    Duplicate records
    Incorrect data types

The presence of these may cause problems when it's time to analyze the data, so we want to make sure we address them beforehand. We can start by visually inspecting the data using the head method, which will show us the first 5 rows of data. 

In [2]:
data.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


# Missing Values

From this initial view, we can see that our data set contains some columns that have missing values in them and others that seem to have a lot of zero values. Let's see how prevalent missing values are in our data. We can use the Pandas isnull method to check whether the value in each field is missing (null) and return either True or False for each field. We can use the sum method to total up the number of True values by column, and then we can add a condition using square brackets that will filter the data and show us only columns where the number of null values were greater than zero. 

In [3]:
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

cylinders       123
displ           120
drive          1189
eng_dscr      15403
trany            11
guzzler       35562
trans_dscr    22796
tCharger      32657
sCharger      37177
atvType       34771
fuelType2     36435
rangeA        36440
evMotor       37281
mfrCode       30818
c240Dscr      37806
c240bDscr     37807
startStop     31705
dtype: int64

In [4]:
type(null_cols)
n = list(null_cols.index)

We can see that some columns have relatively few null values while others have tens of thousands of nulls. For fields that have a lot of null values, you will often have to make a judgement call. If you don't think the information is going to be very useful to your analysis, then you would remove those columns from your data frame. In Pandas, we can do that using the drop method. For our purposes, let's remove the columns that have more than 10,000 null values in them. We will add these column names to a list, and then we will pass those columns to the drop method and indicate that we want columns (not rows) dropped by setting the axis parameter to 1. 

In [5]:
drop_cols = list(null_cols[null_cols > 10000].index)
data = data.drop(drop_cols, axis=1)

In [6]:
data.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,UHighwayA,VClass,year,youSaveSpend,charge240b,createdOn,modifiedOn,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,0.0,Two Seaters,1985,-1250,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,0.0,Two Seaters,1985,-8500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,0.0,Subcompact Cars,1985,500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,0.0,Vans,1985,-8500,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,0.0,Compact Cars,1993,-4000,0.0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,0,0,0


This leaves us with just a handful of remaining columns that have null values. Of the columns that remain, it looks like the cylinders column and the displ column have a similar number of nulls. Perhaps they are missing for similar reasons. We can investigate this by subsetting the data set and looking at just the records where displ is null and just the columns we think will be informative in allowing us to determine a reason. 

In [7]:
null_displ = data[(data['displ'].isnull()==True)]
null_displ = null_displ[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]
null_displ.head()

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,NaN,Electricity,NaN,NaN
7139,2000,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
8143,2001,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
8144,2001,Ford,Th!nk,NaN,NaN,Electricity,NaN,NaN
8146,2001,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,NaN,NaN


We can see that most of the time, cylinders is null when displ is null and that the most of the records where both fields are null have a fuel type of Electricity. This makes sense, as electric cars do not have cylinders and can therefore not have any displacement. In this case, it would make sense to replace these null values with zeros. Pandas makes it easy to do that with the fillna method. 

In [8]:
data[['displ', 'cylinders']] = data[['displ', 'cylinders']].fillna(0)

In this example, we filled the nulls in with zeros, but there are other strategies for filling in nulls. Depending on the circumstances, you might want to replace nulls with the column mean or mode values. Once you get more advanced, you can even use a variety of predictive imputation methods.

Challenge: Now that we have filled those null values in with zeros, there are only two columns in the data set that still have null values: trany and drive. Use what you have learned in this section to investigate and potentially fill in the remaining null values.



# Incorrect Values

In addition to null values, we also want to try to identify any values that seem incorrect. For example, in the previous section, we learned that a vehicle without cylinders should not have displacement and vice versa. Let's check to see if there are any cases that violate these rules.

In [9]:
test = data[(data['cylinders']==0) & (data['displ']!=0)]
test[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

,year,make,model,trany,drive,fuelType,cylinders,displ
21506,1986,Mazda,RX-7,Manual 5-spd,Rear-Wheel Drive,Regular,0.0,1.3


Here we have identified a vehicle with a regular gasoline engine that reportedly does not have any cylinders but does have a value for displacement. The way we would correct this would be to either perform some domain research or ask a domain expert to find out how many actual cylinders this vehicle had. Alternatively, you can also try to look at similar vehicles in the data set and determine the most likely value for this field.

Suppose that using one of the aforementioned methods, we found out that this vehicle actually has a 4 cylinder engine. Once we have this information, we can use the loc method to update that specific value in the data frame. 

In [10]:
data.loc[(data['cylinders']==0) & (data['displ']!=0), 'cylinders'] = 4

 Challenge: Try to find other values that might be incorrect in the data set based on what you know about automobiles and correct them.




# Low Variance Columns

When analyzing data, we want the fields we are working with to be informative, and we will want to strip away any columns that don't have a lot of value to us. One easy way to do this is to identify columns that have low variance, where the majority of the values in the column are the same. Since there is not a lot of variability in these columns, they have the potential to not be as informative as columns that have a variety of different values in them.

Let's try to identify columns where at least 90% of the values are the same so that we can remove them from our data set. To do this, we are going to create an empty list called low_variance that will eventually contain the names of columns that fit our criteria. We will then write a for loop that will take the minimum and the 90th percentile value for all the numeric columns in our data set (identified via the _get_numeric_data method). If the 90th percentile and the minimum are equal to each other, that means that at least 90% of the values in that column are the same and we will append that column name to our low_variance list. 

In [11]:
low_variance = []

for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print(low_variance)


['barrelsA08', 'charge120', 'charge240', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2A', 'co2TailpipeAGpm', 'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'fuelCostA08', 'ghgScoreA', 'highwayA08', 'highwayA08U', 'highwayCD', 'highwayE', 'highwayUF', 'phevBlended', 'range', 'rangeCity', 'rangeCityA', 'rangeHwy', 'rangeHwyA', 'UCityA', 'UHighwayA', 'charge240b', 'phevCity', 'phevHwy', 'phevComb']


This returned 34 columns that we could potentially eliminate due to not having high enough variability to be informative. Of course, before we do this, we should check the values that do exist in these fields to confirm that they are not very informative. Once they have been checked, we can use the the drop method like we did earlier in this lesson to remove those columns from our data frame.



In [12]:
data = data.drop(low_variance, axis=1)

# Extreme Values and Outliers

Now that we have removed low variance columns, we should look for outliers, or extreme values, in the columns that remain. These outliers can influence our aggregations when we are analyzing data later, so we want to make sure we address them during our data cleaning stage.

A common method for identifying outliers is one that leverages the interquartile range (IQR). Once the IQR is calculated, it is multiplied by a constant (typically 1.5) and lower and upper bounds are established at:

    25th Percentile - (IQR x 1.5)
    75th Percentile + (IQR x 1.5)

Any values outside this range are potential outliers and should be investigated.

Let's look at how we would do this for our data set using Python. We will use the Pandas describe function to easily calculate the 25th and 75th percentiles for every column and transpose the results so that we can easily reference the values in calculating the interquartile ranges.



In [23]:
data.describe()

,displ,cylinders,barrels08,city08,highway08,comb08,co2TailpipeGpm,fuelCost08
count,36958.000000,36958.000000,36958.000000,36958.000000,36958.000000,36958.000000,36958.000000,36958.000000
mean,3.313572,5.728773,17.518871,17.950538,24.134017,20.211267,472.831339,1882.593755
std,1.371037,1.781388,4.565909,6.705373,6.990806,6.661346,121.889046,510.085993
min,0.000000,0.000000,0.060000,6.000000,9.000000,7.000000,0.000000,550.000000
25%,2.200000,4.000000,14.330870,15.000000,20.000000,16.000000,387.000000,1500.000000
50%,3.000000,6.000000,17.347895,17.000000,24.000000,19.000000,467.736842,1850.000000
75%,4.300000,6.000000,20.600625,20.000000,27.000000,23.000000,555.437500,2200.000000
max,8.400000,16.000000,47.087143,138.000000,111.000000,124.000000,1269.571429,5800.000000


In [22]:
stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
displ,36958.0,3.313572,1.371037,0.00,2.20000,3.000000,4.300000,8.400000,2.100000
cylinders,36958.0,5.728773,1.781388,0.00,4.00000,6.000000,6.000000,16.000000,2.000000
barrels08,36958.0,17.518871,4.565909,0.06,14.33087,17.347895,20.600625,47.087143,6.269755
city08,36958.0,17.950538,6.705373,6.00,15.00000,17.000000,20.000000,138.000000,5.000000
highway08,36958.0,24.134017,6.990806,9.00,20.00000,24.000000,27.000000,111.000000,7.000000
comb08,36958.0,20.211267,6.661346,7.00,16.00000,19.000000,23.000000,124.000000,7.000000
co2TailpipeGpm,36958.0,472.831339,121.889046,0.00,387.00000,467.736842,555.437500,1269.571429,168.437500
fuelCost08,36958.0,1882.593755,510.085993,550.00,1500.00000,1850.000000,2200.000000,5800.000000,700.000000


We will then create an empty data frame called outliers with the same columns as our data set. Finally, we will loop through each column in the data calculating the lower and upper bounds, retrieving records where the value for that column falls outside the bounds we established, and appending those results to our outlier data frame.

In [24]:
stats.index

Index(['displ', 'cylinders', 'barrels08', 'city08', 'highway08', 'comb08',
       'co2TailpipeGpm', 'fuelCost08'],
      dtype='object')

In [14]:
outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    

outliers.head()

,barrels08,city08,city08U,co2,co2TailpipeGpm,comb08,comb08U,cylinders,displ,drive,...,pv4,trany,UCity,UHighway,VClass,year,youSaveSpend,createdOn,modifiedOn,Outlier
47,32.961,9,0.0,-1,888.7,10,0.0,8.0,5.9,Rear-Wheel Drive,...,0,Automatic 3-spd,11.0,15.0,Vans,1985,-10000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,barrels08
58,32.961,10,0.0,-1,888.7,10,0.0,8.0,5.2,Rear-Wheel Drive,...,0,Automatic 3-spd,12.0,15.0,Vans,1985,-10000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,barrels08
69,32.961,10,0.0,-1,888.7,10,0.0,8.0,5.2,Rear-Wheel Drive,...,0,Automatic 3-spd,12.0,15.0,Vans,1985,-10000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,barrels08
80,32.961,9,0.0,-1,888.7,10,0.0,8.0,5.9,Rear-Wheel Drive,...,0,Automatic 3-spd,11.0,14.0,Vans,1985,-10000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,barrels08
275,32.961,9,0.0,-1,888.7,10,0.0,8.0,7.4,Rear-Wheel Drive,...,0,Automatic 4-spd,11.0,16.0,Standard Pickup Trucks,1993,-10000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,barrels08


Our outliers data frame should now be populated with records that you can investigate further and determine whether they should be kept in the data or dropped. The Outlier column we added before appending the results for the column to the outliers data frame will let you know what column in each record contained the outlier. If you find that this method is returning too many results, you can be more stringent with your cutoff criteria (e.g. increasing the constant by which you multiply the IQR to 3 instead of 1.5).
Data Type Correction

One common problem that is often overlooked is incorrect data types. This typically occurs when there is a numeric variable that should actually be represented as a categorical variable. The way to check the data type of each column in Pandas is by using the dtypes method. 

In [15]:
data.dtypes

barrels08         float64
city08              int64
city08U           float64
co2                 int64
co2TailpipeGpm    float64
comb08              int64
comb08U           float64
cylinders         float64
displ             float64
drive              object
engId               int64
feScore             int64
fuelCost08          int64
fuelType           object
fuelType1          object
ghgScore            int64
highway08           int64
highway08U        float64
hlv                 int64
hpv                 int64
id                  int64
lv2                 int64
lv4                 int64
make               object
model              object
mpgData            object
pv2                 int64
pv4                 int64
trany              object
UCity             float64
UHighway          float64
VClass             object
year                int64
youSaveSpend        int64
createdOn          object
modifiedOn         object
dtype: object

Pandas currently has the year column stored as integers, but what if we wanted the year to be stored as a categorical variable (object) instead? We could easily change that data type using the astype method and then check that it changed using the dtypes method again just on that field. 

In [16]:
data['year'] = data['year'].astype('object')
data['year'].dtype

dtype('O')

You can apply this technique to any column whose data type you would like to change.



# Cleaning Text and Removing Special Characters

The presence of special characters in our fields has the potential to make analyzing our data challenging. Imagine not being able to perform calculations on a numeric field because it was currently represented as an object data type due to the fact that it had a dollar sign ($) in it. Similarly, imagine having a categorical field where you could not group records that belong in the same group together because in one field you are grouping by, terms that refer to the same thing are sometimes hyphenated. In cases like this, it is necessary to remove special characters so that we can properly analyze the data.

In our vehicles data set, the trany field has several special characters (parentheses, hyphens, etc.). We can take a look at the unique values in this column by using the set function. 

In [17]:
print(set(data['trany']))

{nan, 'Automatic 6-spd', 'Auto(AM-S8)', 'Auto(AM6)', 'Automatic (AV)', 'Automatic 9-spd', 'Auto(AV-S7)', 'Manual 5 spd', 'Auto(AM-S9)', 'Manual 3-spd', 'Automatic 8-spd', 'Auto(AV-S8)', 'Automatic (variable gear ratios)', 'Automatic (S4)', 'Auto(AV-S6)', 'Automatic 5-spd', 'Auto(AM-S6)', 'Automatic (S9)', 'Auto(L3)', 'Auto (AV-S6)', 'Auto (AV-S8)', 'Auto(A1)', 'Automatic 3-spd', 'Automatic 7-spd', 'Manual 4-spd Doubled', 'Automatic (AM5)', 'Automatic (S7)', 'Manual 5-spd', 'Automatic (S5)', 'Auto (AV)', 'Automatic 4-spd', 'Automatic (S8)', 'Manual(M7)', 'Auto(AM5)', 'Manual 4-spd', 'Auto(AM-S7)', 'Automatic 6spd', 'Auto(AM8)', 'Automatic (A6)', 'Automatic (A1)', 'Auto(L4)', 'Manual 6-spd', 'Automatic (AV-S6)', 'Automatic (S6)', 'Auto(AM7)', 'Manual 7-spd', 'Automatic (AM6)'}


We can see that there are instances that refer to the same thing, but would not get grouped together due to special characters (e.g. Automatic 6-spd and Automatic 6spd). So let's remove all hyphens from this column with the help of the str.replace method and then print unique values again to ensure they were removed.

In [18]:
data['trany'] = data['trany'].str.replace('-', '')
print(set(data['trany']))

{nan, 'Auto(AVS6)', 'Auto (AVS8)', 'Manual 6spd', 'Manual 3spd', 'Auto(AM6)', 'Automatic 4spd', 'Automatic 5spd', 'Automatic (AV)', 'Manual 4spd Doubled', 'Manual 4spd', 'Automatic 7spd', 'Manual 5 spd', 'Auto(AVS8)', 'Automatic (variable gear ratios)', 'Automatic (S4)', 'Auto(AMS6)', 'Automatic (S9)', 'Auto(L3)', 'Auto(AMS8)', 'Automatic 9spd', 'Auto(A1)', 'Automatic 3spd', 'Automatic (AM5)', 'Automatic (S7)', 'Automatic 8spd', 'Automatic (S5)', 'Auto (AV)', 'Auto(AMS9)', 'Automatic (S8)', 'Manual(M7)', 'Manual 7spd', 'Auto(AM5)', 'Automatic 6spd', 'Auto(AM8)', 'Automatic (A6)', 'Automatic (A1)', 'Manual 5spd', 'Automatic (AVS6)', 'Auto(L4)', 'Auto (AVS6)', 'Automatic (S6)', 'Auto(AM7)', 'Auto(AMS7)', 'Automatic (AM6)', 'Auto(AVS7)'}


You will also notice that in some cases Automatic is abbreviated to Auto and in other cases it is spelled out. We can make that more consistent by using the same technique. While we are at it, let's also attempt to remove parentheses and make spacing more consistent.

In [19]:
data['trany'] = data['trany'].str.replace('Automatic', 'Auto')
data['trany'] = data['trany'].str.replace('Auto\(', 'Auto ')
data['trany'] = data['trany'].str.replace('Manual\(', 'Manual ')
data['trany'] = data['trany'].str.replace('\(', '')
data['trany'] = data['trany'].str.replace('\)', '')
print(set(data['trany']))

{nan, 'Auto 3spd', 'Manual 6spd', 'Manual 3spd', 'Manual 4spd Doubled', 'Auto AM7', 'Manual 4spd', 'Auto 9spd', 'Manual 5 spd', 'Auto AVS7', 'Auto 4spd', 'Auto 7spd', 'Auto S7', 'Auto 8spd', 'Auto S9', 'Auto AVS6', 'Auto AV', 'Auto AMS8', 'Auto L4', 'Auto A6', 'Auto S8', 'Auto variable gear ratios', 'Auto L3', 'Auto 5spd', 'Auto S4', 'Manual M7', 'Auto AM6', 'Auto AM8', 'Auto AMS7', 'Auto AVS8', 'Auto AM5', 'Auto S5', 'Auto 6spd', 'Auto AMS6', 'Manual 5spd', 'Auto S6', 'Auto AMS9', 'Auto A1', 'Manual 7spd'}


<ipython-input-19-b7c50a551c79>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['trany'] = data['trany'].str.replace('Auto\(', 'Auto ')
<ipython-input-19-b7c50a551c79>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['trany'] = data['trany'].str.replace('Manual\(', 'Manual ')
<ipython-input-19-b7c50a551c79>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['trany'] = data['trany'].str.replace('\(', '')
<ipython-input-19-b7c50a551c79>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['trany'] = data['trany'].str.replace('\)', '')


As you can see, we now have no special characters, consistent naming, and proper spacing. We started out with 47 unique values in this column, and using this technique, we were able to reduce the number of unique values to 39.



# Finding and Removing Duplicates

The final topic we are going to cover in this lesson is how to identify and remove duplicate rows (or rows that refer to the same entity) in our data. When trying to identify duplicates, we will use the columns (or attributes) of the data to help us determine what entities are similar enough to be considered the same entity. We want to start with all the columns we currently have available to us and work our way toward a lesser number of attributes in an intuitive fashion. In this process, the act of dropping duplicated records is easy, but identifying the correct attributes for comparison and which records to drop is sometimes quite challenging.

The first thing we will do is attempt to drop any duplicate records, considering all the columns we currently have in the data set. Pandas provides us with the ability to do that via the drop_duplicates method. We will use the len method to calculate the number of rows in the data set both before and after removing duplicates and then print the number of rows dropped. 

In [20]:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


This tells us that there were no records that matched exactly across all columns. However, if we reduce the number of columns in our data that we are interested in, we can try again and have a higher likelihood of finding duplicate records. In the example below, we will select a subset of columns, remove all other columns, and then use the drop_duplicates method to drop any duplicate records based on the remaining columns. 

In [21]:
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 
                  'trany', 'drive', 'VClass','fuelType','barrels08', 
                  'city08', 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']

data = data[select_columns].drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  885


With the reduced number of columns, we were able to identify and drop 885 duplicate records.